### Testing Speech to Text (STT) using OpenSLR dataset with Javanese language

#### Works, but not good enough. Need to find better dataset and training.

In [8]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np

In [9]:
MODEL_LOAD = './model/modelJava235-CTC-TF.h5'
SOUND_PATH = './assets/Recording4Edited.wav'

In [10]:
# predict the sound in SOUND_PATH using the model in MODEL_PATH
frameLength = 256
frameStep = 160
fftLength = 384

def predictSound(modelPath, soundPath):
    model = keras.models.load_model(modelPath, compile=False)
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = 1e-4), loss = CTCLoss)
    file = tf.io.read_file(soundPath)
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    audio = tf.cast(audio, tf.float32)

    spectogram = tf.signal.stft(audio, frame_length=frameLength, frame_step=frameStep, fft_length=fftLength)
    spectogram = tf.abs(spectogram)
    spectogram = tf.math.pow(spectogram, 0.5)

    means = tf.math.reduce_mean(spectogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectogram, 1, keepdims=True)
    spectogram = (spectogram - means) / (stddevs + 1e-10)

    spectogram = tf.expand_dims(spectogram, axis=0)
    prediction = model.predict(spectogram)
    prediction = decodeBatchPredictions(prediction)[0]
    return prediction

In [11]:
def decodeBatchPredictions(pred):
    inputLen = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred, input_length=inputLen, greedy=True)[0][0]
    outputText = []
    for result in results:
        result = tf.strings.reduce_join(numToChar(result)).numpy().decode('utf-8')
        outputText.append(result)
    return outputText

In [12]:
def CTCLoss (y_true, y_pred):
    batchLen = tf.cast(tf.shape(y_true)[0], dtype='int64')
    inputLen = tf.cast(tf.shape(y_pred)[1], dtype='int64')
    labelLen = tf.cast(tf.shape(y_true)[1], dtype='int64')

    inputLen = inputLen * tf.ones(shape=(batchLen, 1), dtype='int64')
    labelLen = labelLen * tf.ones(shape=(batchLen, 1), dtype='int64')

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, inputLen, labelLen)
    return loss

In [13]:
char = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]
charToNum = keras.layers.StringLookup(vocabulary=char, oov_token="")
numToChar = keras.layers.StringLookup(vocabulary=charToNum.get_vocabulary(), oov_token="", invert=True)

print(f"The Vocabulary size: {charToNum.get_vocabulary()}")
print(f"(size = {charToNum.vocabulary_size()})")

The Vocabulary size: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', ' ']
(size = 31)


In [14]:
predictSound(MODEL_LOAD, SOUND_PATH)

1/1 [==============================] - 3s 3s/step


'aaora daal are eakeacedhe aaiakida'